# Lab | APIs

In order to use the `Spotify` API (`SpotiPy`), create an account in `Spotify` and follow [these](https://developer.spotify.com/documentation/general/guides/app-settings/) steps. 

## Authentication and initializing the API

Save your client ID and your client secret in your preferred way, and read it or load it into the following variables:

In [1]:
import requests
import pandas as pd

In [2]:
CLIENT_ID = "<introduce your client id>"
CLIENT_SECRET = "<introduce your client secret>"

In [3]:
CLIENT_ID = "9f8fa04c80a94cbc99b0190449fe7374"
CLIENT_SECRET = "0169432afd76420a8b992fd89788610b"

In [4]:
# If you havent done so, install the spotipy wrapper
!pip install spotipy

Once you have done it, we will start initializing the API.

In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                           client_secret=CLIENT_SECRET))


## Exercise 1: Discovering New Music through Your Favorite Artists

**Objective:** 
Uncover new music by exploring the top tracks of your favorite artists and their related artists.

**Instructions:**

1. **List Your Favorite Artists**:
    - Make a list of your three favorite artists and store it in a variable named `artists`.
    - Example: `artists = ["Los Fabulosos Cadillacs", "Manu Chao", "Muchachito Bombo Infierno"]`.

2. **Fetch Top Tracks**:
    - Write a function named `get_top_tracks`.
    - This function should accept an artist's name and return the name of the first 5 top tracks by that artist.
    - Use the function `get_top_tracks` to get the first 5 top tracks for each artist in your `artists` list and store the results in a new list named `top_tracks_list`.

3. **Discover Related Artists**:
    - Write a function named `find_related_artists`.
    - This function should accept an artist's name and return the names of the first 5 artists related to the provided artist.
    - Store the results in a list named `related_artists_list`.

**Challenge:** 
Combine the above steps to create a playlist that includes the top tracks of your favorite artists and the top tracks of the artists related to them.

**Hint Section for 3. **Discover Related Artists**:**

1. **Getting Artist ID**:
    - Remember that every artist on Spotify has a unique identifier: their `id`. To get the related artists, you first need to fetch the ID of the given artist.
    - Consider using the `sp.search` method to query the artist's name. The method requires a `q` parameter, which is your query (in this case, the artist's name). It also has a `limit` parameter, which specifies the number of tracks it returns. In this case, 1 track is enough, since we just want the artist ID. 
    - Each track in the results has an associated 'artists' field. This field is a list containing details about all artists involved in that track.
   - For most tracks, especially those by a single artist, this list will contain one artist. From this artist's details, you can extract the 'id' field, which is the unique identifier for that artist on Spotify.


3. **Fetching Related Artists**:
    - Once you have the artist's ID, you can use another SpotiPy method to fetch related artists. Think about which SpotiPy method allows you to get related artists using an artist's ID. Here is the documentation link: https://spotipy.readthedocs.io/en/2.22.1/. 
    - This method will return a list of related artists. You'll need to extract the relevant details (artist names) from this list.

4. **Iterating for Multiple Artists**:
    - Once you have a function that returns related artists names for one artist, you can use a list comprehension to apply this function to a list of artist names.

5. **Testing**:
    - Always test your function with one artist name first. Once you're confident it works, then apply it to the entire list.

Remember, the key is to break the problem down into manageable steps. Use the SpotiPy documentation as a resource to understand available methods and their return structures.

In [6]:
# Exercise 1: Discovering New Music through Your Favorite Artists

# 1. List Your Favorite Artists:
artists = ["Fred again..", "The Lumineers", "Sen Senra"]

In [7]:
# Get the first artist results
results_fredagain = sp.search(q='Fred Again..', limit=50)

In [8]:
# Get the second artist results
results_thelumineers = sp.search(q='The Lumineers', limit=50)

In [9]:
# Get the third artist results
results_sensenra = sp.search(q='Sen Senra', limit=50)

In [10]:
# 2. Fetch Top Tracks:
def get_top_tracks(artist_data):
    top_tracks_list = []
    # Verificar si el diccionario contiene información sobre las pistas
    if artist_data and "tracks" in artist_data and "items" in artist_data["tracks"]:
        # Iterar sobre las pistas y obtener los nombres de las primeras 5
        for track in artist_data["tracks"]["items"][:5]:
            track_name = track["name"]
            top_tracks_list.append(track_name)
        
    return top_tracks_list

In [11]:
# Call the function for the first artist
get_top_tracks(results_fredagain)

['adore u',
 'Delilah (pull me out of this)',
 'ten',
 'Jungle',
 'BerwynGesaffNeighbours']

In [12]:
# Call the function for the second artist
get_top_tracks(results_thelumineers)

['Ho Hey',
 'Spotless (feat. The Lumineers)',
 'Ophelia',
 'Flowers in Your Hair',
 'Sleep On The Floor']

In [13]:
# Call the function for the third artist
get_top_tracks(results_sensenra)

['1000CANCIONES',
 '86.400',
 'Nada Y Nadie',
 'Ya No Te Hago Falta',
 'Uno De Eses Gatos']

In [14]:
# 3. Discover Related Artists:

def find_related_artists(artist_name):
    related_artists_list = []
    
    results = sp.search(q=artist_name, limit=1)
    
    if results['tracks']['items']:
        artist_id = results['tracks']['items'][0]['artists'][0]['id']
        
        related_artists = sp.artist_related_artists(artist_id)
        
        for artist in related_artists['artists'][:5]:
            related_artists_list.append(artist['name'])
    
    return related_artists_list
        
        
        

In [15]:
# Call the function for the first artist
find_related_artists("Fred Again..")

["Barry Can't Swim", 'Dom Dolla', 'BICEP', 'John Summit', 'FISHER']

In [16]:
# Call the function for the second artist
find_related_artists("The Lumineers")

['Of Monsters and Men',
 'Vance Joy',
 'The Head And The Heart',
 'Mumford & Sons',
 'Edward Sharpe & The Magnetic Zeros']

In [17]:
# Call the function for the third artist
find_related_artists("Sen Senra")

['legallyrxx', 'Papi Sousa', 'Nsqk', 'ALVAR0 DAYS', 'Tainy']

In [18]:
# Challenge: Combine the above steps to create a playlist that includes the top tracks of your favorite artists and the top tracks of the artists related to them.

# Lista para almacenar las principales pistas de todos los artistas
playlist_tracks = []

# Iterar sobre los artistas favoritos
for artist_name in artists:
    # Obtener información sobre el artista actual
    artist_data = sp.search(q=artist_name, limit=1)
    
    # Obtener las principales pistas del artista actual
    top_tracks = get_top_tracks(artist_data)
    # Agregar las principales pistas del artista actual a la lista de reproducción
    playlist_tracks.extend(top_tracks)
    
    # Obtener los artistas relacionados del artista actual
    related_artists = find_related_artists(artist_name)
    # Iterar sobre los artistas relacionados
    for related_artist_name in related_artists:
        # Obtener información sobre el artista relacionado
        related_artist_data = sp.search(q=related_artist_name, limit=1)
        # Obtener las principales pistas del artista relacionado
        top_tracks_related = get_top_tracks(related_artist_data)
        # Agregar las principales pistas del artista relacionado a la lista de reproducción
        playlist_tracks.extend(top_tracks_related)

# Imprimir la lista de reproducción
for idx, track_name in enumerate(playlist_tracks, 1):
    print(f"{idx}. {track_name}")

1. leavemealone
2. How It Feels
3. Take It
4. Glue
5. Shiver
6. TAKE IT OFF
7. Ho Hey
8. Little Talks
9. Riptide
10. Rivers and Roads
11. I Will Wait
12. Home
13. 1000CANCIONES
14. Weeknd<3
15. 1%
16. EN EL GHETTO #3 (Par de cosas)
17. Sabor a Ti
18. MOJABI GHOST


## Exercise 2: Unraveling the World of Playlists


1. **Featured Exploration**: 
   - Fetch the list of Spotify's current featured playlists. 
   - Extract and display the names and IDs of the top 5 featured playlists.
   
2. **Deep Dive**:
   - Choose any one of the top 5 featured playlists (you can choose the one you personally find most interesting or simply pick one randomly).
   - Fetch and display its name, description, and total track count.

3. **Track-tastic**:
   - Extract and display the names of the first 10 tracks in the chosen playlist.

4. **Artistic Flair**:
   - Create a dictionary where the keys are the names of the first 10 tracks, and the values are lists containing the names of the artists associated with each track.
   - For example: `{"TrackName1": ["Artist1", "Artist2"], "TrackName2": ["Artist3"]}`
   

In [19]:
# 1.Featured Exploration:
# Fetch the list of Spotify's current featured playlists
# Extract and display the names and IDs of the top 5 featured playlists.

def fetch_top_featured_playlists():
    featured_playlists = sp.featured_playlists()
    
    if 'playlists' in featured_playlists and 'items' in featured_playlists['playlists']:
        print("Top 5 Featured Playlists:")
        for idx, playlist in enumerate(featured_playlists['playlists']['items'][:5], 1):
            playlist_name = playlist['name']
            playlist_id = playlist['id']
            print(f"{idx}. Name: {playlist_name}, ID: {playlist_id}")
    else:
        print("No featured playlists found.")

fetch_top_featured_playlists()

Top 5 Featured Playlists:
1. Name: Éxitos España, ID: 37i9dQZF1DXaxEKcoCdWHD
2. Name: PEGAO, ID: 37i9dQZF1DX1HCSfq0nSal
3. Name: Viva Latino, ID: 37i9dQZF1DX10zKzsJ2jva
4. Name: Viral España 2024, ID: 37i9dQZF1DWVJv1UsWItkB
5. Name: míticas, ID: 37i9dQZF1DX2EiUAL7Wdfc


In [20]:
# 2. Deep Dive:
# Choose any one of the top 5 featured playlists (you can choose the one you personally find most interesting or simply pick one randomly).
# Fetch and display its name, description, and total track count.

def fetch_playlist_info(playlist_id):

    playlist_info = sp.playlist(playlist_id)
    
    playlist_name = playlist_info['name']
    playlist_description = playlist_info['description']
    playlist_track_count = playlist_info['tracks']['total']
    
    print(f"Playlist Name: {playlist_name}")
    print(f"Description: {playlist_description}")
    print(f"Total Tracks: {playlist_track_count}")

playlist_id = '37i9dQZF1DX1HCSfq0nSal'
fetch_playlist_info(playlist_id)

Playlist Name: PEGAO
Description: Si está pegao, está aquí. Feat. SAIKO
Total Tracks: 40


In [21]:
# 3. Track-tastic:
#Extract and display the names of the first 10 tracks in the chosen playlist.

tracks = sp.playlist_tracks(playlist_id)
for track in tracks['items']:
    print(track['track']['name'])  

BADGYAL
ADIVINO
Santa
BBY BOO - REMIX
Kilerito
LA NENA
LA RANGER (feat. Myke Towers)
YO LO SOÑÉ
FRIKI
Espectacular
Bien Loco
Guay
El Conjuntito
Martini
SI SI SI SI (feat. The Academy: Segunda Misión)
La Vida Sin Ti
No Te Quieren Conmigo - Remix
NÚMERO TELEFÓNICO
Diabólica
DESATAAA
BESAME (feat. Tiago PZK, Khea & Neo Pistea) - Remix
No Digas Na
X'CLUSIVO - REMIX
Lo Que Tiene
offline
CRUSH
Trending Remix (with Myke Towers)
SHE TELLS ME
TU$$I (with Dei V)
Gata Only
PERREO LENTO
CARNET
BAD BOY
EMPELOTICA
CORLEONE
SI SABE FERXXO
Hey Lil Mama
Una Foto Remix (feat. Emilia)
FERXXO 151
DILUVIO


In [22]:
# 4. Artistic Flair:
#Create a dictionary where the keys are the names of the first 10 tracks, and the values are lists containing the names of the artists associated with each track.

def create_tracks_dict():
    featured_playlists = sp.featured_playlists()

    if 'playlists' in featured_playlists and 'items' in featured_playlists['playlists']:
        first_playlist = featured_playlists['playlists']['items'][0]
        tracks_info = sp.playlist_tracks(first_playlist['id'], limit=10)
        
        tracks_dict = {}
        for track_info in tracks_info['items']:
            track_name = track_info['track']['name']
            artists = [artist['name'] for artist in track_info['track']['artists']]
            tracks_dict[track_name] = artists
        
        return tracks_dict
    else:
        print("No featured playlists found.")
        return {}

tracks_dict = create_tracks_dict()

print(tracks_dict)

{'YO LO SOÑÉ': ['SAIKO', 'Omar Montes'], 'Santa': ['Rvssian', 'Rauw Alejandro', 'Ayra Starr'], 'ADIVINO': ['Myke Towers', 'Bad Bunny'], 'El Conjuntito': ['El Bobe', 'Omar Montes'], 'BADGYAL': ['SAIKO', 'JC Reyes'], 'LA SEVILLANA': ['Omar Montes'], "X'CLUSIVO - REMIX": ['Gonzy', 'SAIKO', 'Arcángel'], 'Tengo Un Plan': ['Key-Key'], 'NANA DEL HILO ROJO': ['SAIKO', 'Sky Rompiendo'], 'Lo Que Tiene': ['Morad', 'Beny Jr', 'Rvfv']}
